In [ ]:
using CSV
using Dates
using DataFramesMeta

using PlotlyJS

basepath = "../data"
include("storageOptimization.jl");
include("$basepath/datacatalog.jl");

In [2]:
prices = reduce(
    vcat,
    [
        CSV.read("$basepath/MISO DA Energy Price by Select Nodes 2023.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("$basepath/MISO DA Energy Price by Select Nodes 2024.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
    ]
)

gre = @rsubset prices :node == "GRE.ALTW";
nobles = @rsubset prices :node == "NSP.NOBLES.WND";
pvaley = @rsubset prices :node == "NSP.PVALEY.WND";
minnhub = @rsubset prices :node == "MINN.HUB";

pvaley.lmp = coalesce.(pvaley.lmp, 22.19);
nobles.lmp = coalesce.(nobles.lmp, 16.945);
disallowmissing!(gre, :lmp);
disallowmissing!(nobles, :lmp);
disallowmissing!(pvaley, :lmp);
disallowmissing!(minnhub, :lmp);	

In [3]:
# prices = reduce(
#     vcat,
#     [
#         CSV.read("$basepath/CAISO DA LMP NP15 2020-2024.csv", DataFrame; dateformat=caisoDAEnergyPrice.dateformat, types=caisoDAEnergyPrice.schema, validate=true),
#         CSV.read("$basepath/CAISO DA LMP SP15 2020-2024.csv", DataFrame; dateformat=caisoDAEnergyPrice.dateformat, types=caisoDAEnergyPrice.schema, validate=true),
#     ]
# )

# np15 = @chain prices begin
#     @rsubset :location == "TH_NP15_GEN-APND"
#     @rsubset :interval_start_local >= ZonedDateTime("2023-01-01T00:00:00-08:00","yyyy-mm-ddTHH:MM:SSzzzz")
#     @rsubset :interval_start_local < ZonedDateTime("2025-01-01T00:00:00-08:00","yyyy-mm-ddTHH:MM:SSzzzz")
# end;

# sp15 = @chain prices begin
#     @rsubset :location == "TH_SP15_GEN-APND"
#     @rsubset :interval_start_local >= ZonedDateTime("2023-01-01T00:00:00-08:00","yyyy-mm-ddTHH:MM:SSzzzz")
#     @rsubset :interval_start_local < ZonedDateTime("2025-01-01T00:00:00-08:00","yyyy-mm-ddTHH:MM:SSzzzz")
# end;

In [4]:
# prices = reduce(
#     vcat,
#     [
#         CSV.read("$basepath/ERCOT DA SPP HB_HUBAVE 2020-2024.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
#     ]
# )

# hbhub = @chain prices begin
#     @rsubset :interval_start_local >= ZonedDateTime("2023-01-01T00:00:00-05:00","yyyy-mm-ddTHH:MM:SSzzzz")
#     @rsubset :interval_start_local < ZonedDateTime("2025-01-01T00:00:00-05:00","yyyy-mm-ddTHH:MM:SSzzzz")
# end;

In [5]:
configs = Dict(
    # 4hr
    "config-4HR-CAES" => StorageModelConfig(4, 1, roundTripEfficency=55),
    "config-4HR-Lead Acid" => StorageModelConfig(4, 1, roundTripEfficency=77),
    "config-4HR-Lithium-ion LFP" => StorageModelConfig(4, 1, roundTripEfficency=83),
    "config-4HR-Lithium-ion NMC" => StorageModelConfig(4, 1, roundTripEfficency=83),
    "config-4HR-PSH" => StorageModelConfig(4, 1, roundTripEfficency=80),
    "config-4HR-Thermal" => StorageModelConfig(4, 1, roundTripEfficency=48),
    "config-4HR-Vanadium Redox Flow" => StorageModelConfig(4, 1, roundTripEfficency=65),
    "config-4HR-Iron Air" => StorageModelConfig(4, 1, roundTripEfficency=40),

    # 10hr
    "config-10HR-CAES" => StorageModelConfig(10, 1, roundTripEfficency=55),
    "config-10HR-Lead Acid" => StorageModelConfig(10, 1, roundTripEfficency=77),
    "config-10HR-Lithium-ion LFP" => StorageModelConfig(10, 1, roundTripEfficency=83),
    "config-10HR-Lithium-ion NMC" => StorageModelConfig(10, 1, roundTripEfficency=83),
    "config-10HR-PSH" => StorageModelConfig(10, 1, roundTripEfficency=80),
    "config-10HR-Thermal" => StorageModelConfig(10, 1, roundTripEfficency=48),
    "config-10HR-Vanadium Redox Flow" => StorageModelConfig(10, 1, roundTripEfficency=65),
    "config-10HR-Iron Air" => StorageModelConfig(10, 1, roundTripEfficency=40),

    # 24hr
    "config-24HR-CAES" => StorageModelConfig(24, 1, roundTripEfficency=55),
    "config-24HR-Lead Acid" => StorageModelConfig(24, 1, roundTripEfficency=77),
    "config-24HR-Lithium-ion LFP" => StorageModelConfig(24, 1, roundTripEfficency=83),
    "config-24HR-Lithium-ion NMC" => StorageModelConfig(24, 1, roundTripEfficency=83),
    "config-24HR-PSH" => StorageModelConfig(24, 1, roundTripEfficency=80),
    "config-24HR-Thermal" => StorageModelConfig(24, 1, roundTripEfficency=48),
    "config-24HR-Vanadium Redox Flow" => StorageModelConfig(24, 1, roundTripEfficency=65),
    "config-24HR-Iron Air" => StorageModelConfig(24, 1, roundTripEfficency=40),

    # 100hr
    "config-100HR-CAES" => StorageModelConfig(100, 1, roundTripEfficency=55),
    "config-100HR-Lead Acid" => StorageModelConfig(100, 1, roundTripEfficency=77),
    "config-100HR-Lithium-ion LFP" => StorageModelConfig(100, 1, roundTripEfficency=83),
    "config-100HR-Lithium-ion NMC" => StorageModelConfig(100, 1, roundTripEfficency=83),
    "config-100HR-PSH" => StorageModelConfig(100, 1, roundTripEfficency=80),
    "config-100HR-Thermal" => StorageModelConfig(100, 1, roundTripEfficency=48),
    "config-100HR-Vanadium Redox Flow" => StorageModelConfig(100, 1, roundTripEfficency=65),
    "config-100HR-Iron Air" => StorageModelConfig(100, 1, roundTripEfficency=40),
);

In [6]:
technologies = ["Vanadium Redox Flow", "Iron Air"] #"CAES", "Lead Acid", "Lithium-ion LFP", "Lithium-ion NMC", "PSH", "Thermal",;
locations = Dict("minnhub" => minnhub.lmp, "nobles" => nobles.lmp) #, "pvaley" => pvaley.lmp, "gre" => gre.lmp)
# locations = Dict("sp15" => sp15.lmp, "np15" => np15.lmp);
# locations = Dict("hbhub" => hbhub.spp);
durations = [4, 10, 24, 100]
lookaheadPeriods = [24, 48, 72, 96, 168, 240];

In [7]:
cases = Vector{}()
for k in keys(locations)
    for dur in durations
        for tech in technologies
            for lhp in lookaheadPeriods
                push!(cases, Dict(
                    "technology" => tech, 
                    "duration" => dur, 
                    "location" => k,
                    "lookahead" => lhp,
                    "revenue" => 0
                ))
            end
        end
    end
end

In [8]:
for c in cases
    loc = locations[c["location"]]
    dur = c["duration"]
    tech = c["technology"]
    lhp = c["lookahead"]
    r = solveRollingStorageOptimization(loc, configs["config-$(dur)HR-$(tech)"], 24, lhp).revenue / 2
    c["revenue"] = r
end

In [9]:
DataFrame(cases)
CSV.write("storage-revenue-by-tech-duration-node-lookahead.csv", cases)

"storage-revenue-by-tech-duration-node-lookahead.csv"